<a href="https://colab.research.google.com/github/vitorhugoma/Netflix-DataEngineer-Workshop/blob/main/Netflix_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Exploration

---



In this notebook we'll look at one of the first elements involved in any Data Engineering project - Getting to know what the inputs might look like.


---



### Step 1 - What inputs do you have?

Select on your machine the files that you will need for this task.

In [5]:
from google.colab import files
uploaded = files.upload()

Saving WDICountry.csv to WDICountry.csv
Saving WDIData.csv to WDIData.csv
Saving WDISeries.csv to WDISeries.csv


### Step 2 - Get your tools setup

As part of this we will be using PySpark to inspect the data on hand and also gather some basic details.

In [9]:
import os
from IPython.display import display, HTML
import pandas as pd

#Locating where pysp ark is installed
import findspark
findspark.init()
import pyspark

#Settings for PySpark to work
driver_memory = '4g'
num_executors = 2
executor_memory = '1g'
#pyspark_submit_args = ' --driver-memory ' + driver_memory + ' --executor-memory ' + executor_memory + ' --num-executors ' + num_executors + ' pyspark-shell'
pyspark_submit_args = ' --driver-memory ' + driver_memory + ' pyspark-shell'

#Setting the required parameters to start up PySpark
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#Import Modules Needed for PySpark
from pyspark.sql import SparkSession

In [10]:
#Helper for pretty formatting for Spark DataFrames
def showDF(df, limitRows =  20, truncate = True):
    if(truncate):
        pd.set_option('display.max_colwidth', 50)
    else:
        pd.set_option('display.max_colwidth', -1)
    pd.set_option('display.max_rows', limitRows)
    display(df.limit(limitRows).toPandas())
    pd.reset_option('display.max_rows')

In [11]:
#Creating a spark session
spark = SparkSession.builder.appName("Data Exploration").getOrCreate()

### Step 3 - Look inside your data

We need to loog at how our data is composed:

Format - Structure - Size - Dimensions

In this example our input is a CSV file with a header. Let's try to see what the data looks like

#### Read the data

In [13]:
#Read the file into a Spark Data Frame
country = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("WDICountry.csv")

#### Inspect the schema of the file you just read

In [14]:
country.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Short Name: string (nullable = true)
 |-- Table Name: string (nullable = true)
 |-- Long Name: string (nullable = true)
 |-- 2-alpha code: string (nullable = true)
 |-- Currency Unit: string (nullable = true)
 |-- Special Notes: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Income Group: string (nullable = true)
 |-- WB-2 code: string (nullable = true)
 |-- National accounts base year: string (nullable = true)
 |-- National accounts reference year: integer (nullable = true)
 |-- SNA price valuation: string (nullable = true)
 |-- Lending category: string (nullable = true)
 |-- Other groups: string (nullable = true)
 |-- System of National Accounts: string (nullable = true)
 |-- Alternative conversion factor: string (nullable = true)
 |-- PPP survey year: string (nullable = true)
 |-- Balance of Payments Manual in use: string (nullable = true)
 |-- External debt Reporting status: string (nullable = true)
 |-- Sys

##### Take a look at some sample data

You can run .show() to look at the sample data. However the output is not well formatted so we will use our helper function to look at the data.

In [15]:
showDF(country, truncate = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Country Code,Short Name,Table Name,Long Name,2-alpha code,Currency Unit,Special Notes,Region,Income Group,WB-2 code,...,Government Accounting concept,IMF data dissemination standard,Latest population census,Latest household survey,Source of most recent Income and expenditure data,Vital registration complete,Latest agricultural census,Latest industrial data,Latest trade data,Latest water withdrawal data
0,ABW,Aruba,Aruba,Aruba,AW,Aruban florin,SNA data for 2000-2011 are updated from official government statistics; 1994-1999 from UN databases. Base year has changed from 1995 to 2000.,Latin America & Caribbean,High income,AW,...,None,Enhanced General Data Dissemination System (e-GDDS),2010,None,None,Yes,None,NaN,2016.0,NaN
1,AFG,Afghanistan,Afghanistan,Islamic State of Afghanistan,AF,Afghan afghani,"Fiscal year end: March 20; reporting period for national accounts data is calendar year, estimated to insure consistency between national accounts and fiscal data. National accounts data are sourced from the IMF and differ from the Central Statistics Organization numbers due to exclusion of the opium economy.",South Asia,Low income,AF,...,Consolidated central government,Enhanced General Data Dissemination System (e-GDDS),1979,"Demographic and Health Survey, 2015","Integrated household survey (IHS), 2011",None,None,NaN,2016.0,2000.0
2,AGO,Angola,Angola,People's Republic of Angola,AO,Angolan kwanza,None,Sub-Saharan Africa,Lower middle income,AO,...,Budgetary central government,Enhanced General Data Dissemination System (e-GDDS),2014,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2008/09",None,None,NaN,2016.0,2005.0
3,ALB,Albania,Albania,Republic of Albania,AL,Albanian lek,None,Europe & Central Asia,Upper middle income,AL,...,Consolidated central government,Enhanced General Data Dissemination System (e-GDDS),2011,"Demographic and Health Survey, 2008/09","Living Standards Measurement Study Survey (LSMS), 2012",Yes,2012,2013.0,2016.0,2006.0
4,AND,Andorra,Andorra,Principality of Andorra,AD,Euro,WB-3 code changed from ADO to AND to align with ISO code.,Europe & Central Asia,High income,AD,...,None,None,2011. Population data compiled from administrative registers.,None,None,Yes,None,NaN,NaN,NaN
5,ARB,Arab World,Arab World,Arab World,1A,None,Arab World aggregate. Arab World is composed of members of the League of Arab States.,None,None,1A,...,None,None,None,None,None,None,None,NaN,2016.0,NaN
6,ARE,United Arab Emirates,United Arab Emirates,United Arab Emirates,AE,U.A.E. dirham,None,Middle East & North Africa,High income,AE,...,Consolidated central government,Enhanced General Data Dissemination System (e-GDDS),2010,"World Health Survey, 2003",None,None,2012,1985.0,2016.0,2005.0
7,ARG,Argentina,Argentina,Argentine Republic,AR,Argentine peso,"National Institute of Statistics and Census revised national accounts from 2004-2015. Argentina, which was temporarily unclassified in July 2016 pending release of revised national accounts statistics, is classified as upper middle income for FY17 as of September 29, 2016.",None,None,None,...,None,None,None,None,None,None,None,NaN,NaN,NaN
8,ARM,Armenia,Armenia,Republic of Armenia,AM,Armenian dram,None,Europe & Central Asia,Lower middle income,AM,...,Consolidated central government,Special Data Dissemination Standard (SDDS),2011,"Demographic and Health Survey, 2015/16","Integrated household survey (IHS), 2015",Yes,2014,NaN,2016.0,2012.0
9,ASM,American Samoa,American Samoa,American Samoa,AS,U.S. dollar,New base Year 2009,East Asia & Pacific,Upper middle income,AS,...,None,None,2010,None,None,Yes,2008,NaN,2016.0,NaN


##### Get Some Basic Stats

In [16]:
#Count the number of records in the dataframe
country.count()

263

##### Examining Dimensions
How many different regions do the various countries belong to ?

In [17]:
showDF(country.select('Region').distinct(), truncate = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Region
0,South Asia
1,None
2,Sub-Saharan Africa
3,Europe & Central Asia
4,North America
5,East Asia & Pacific
6,Middle East & North Africa
7,Latin America & Caribbean


##### How many different income groups do we have across countries?

In [18]:
showDF(country.select('Income Group').distinct(), truncate = False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


,Income Group
0,Lower middle income
1,None
2,High income
3,Upper middle income
4,Low income


### Step 4 - Transform the data

Transform the Country Dataset

 - Select the columns that we will need for our data model
 - Rename columns for data ingestion

 We'll be using the various operations supported for a DataFrame.


In [26]:
countryDim = country \
    .select("2-alpha code", "Country Code", "Short Name", "Long Name", "Region", "Income Group") \
    .withColumnRenamed("2-alpha code", "country_iso_code") \
    .withColumnRenamed("Country Code", "wb_country_code") \
    .withColumnRenamed("Short Name", "country_name") \
    .withColumnRenamed("Long Name", "country_long_name") \
    .withColumnRenamed("Region", "region") \
    .withColumnRenamed("Income Group", "income_group")
    
showDF(countryDim)

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


You can also do the similar transformations in sql

Let's rename the country_name and country_long_name columns

In [27]:
#Lets you create a view that you can use in SQL queries
countryDim.createOrReplaceTempView("country")

In [28]:
transformQuery = """
select 
    country_iso_code
    , wb_country_code
    , country_name as name
    , country_long_name as long_name
    , region
    , income_group
from 
    country
"""

showDF(spark.sql(transformQuery))

,country_iso_code,wb_country_code,name,long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


### Step 5 - Check the data quality

Do all countries have 2 character country_iso_code?

We will use some sql functions supported by PySpark.

In [34]:
from pyspark.sql import functions as F
showDF( \
    countryDim \
       .select(F.col("country_iso_code"), \
               F.length(F.col("country_iso_code")).alias("column_length")) \
       .groupBy("column_length") \
       .agg(F.count("*").alias("cnt")) \
       .filter("cnt > 1") \
)

,column_length,cnt
0,2,262


In [35]:
countryCodeLengthQuery = """
select 
    length(country_iso_code) as column_length
    , count(1) as cnt
from 
    country
group by 
    length(country_iso_code)
having 
    count(1) > 1
"""

showDF(spark.sql(countryCodeLengthQuery))

,column_length,cnt
0,2,262


### Step 6 - Fix any issues with data quality

In [36]:
countryDimFinal = countryDim.filter("country_iso_code is not null")

showDF(countryDimFinal)

,country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
0,AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
1,AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
2,AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
3,AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
4,AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
5,1A,ARB,Arab World,Arab World,None,None
6,AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
7,AR,ARG,Argentina,Argentine Republic,None,None
8,AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
9,AS,ASM,American Samoa,American Samoa,East Asia & Pacific,Upper middle income


In [37]:
countryDimFinal.count()

262

### Step 7 - Write the data to the destination

In [38]:
countryDimFinal \
    .coalesce(1) \
    .write.csv('CountryDim', mode='overwrite', header='true')

### Step 8 - Check if the output was written out as expected

In [39]:
%%bash
cat CountryDim/*csv | head

country_iso_code,wb_country_code,country_name,country_long_name,region,income_group
AW,ABW,Aruba,Aruba,Latin America & Caribbean,High income
AF,AFG,Afghanistan,Islamic State of Afghanistan,South Asia,Low income
AO,AGO,Angola,People's Republic of Angola,Sub-Saharan Africa,Lower middle income
AL,ALB,Albania,Republic of Albania,Europe & Central Asia,Upper middle income
AD,AND,Andorra,Principality of Andorra,Europe & Central Asia,High income
1A,ARB,Arab World,Arab World,,
AE,ARE,United Arab Emirates,United Arab Emirates,Middle East & North Africa,High income
AR,ARG,Argentina,Argentine Republic,,
AM,ARM,Armenia,Republic of Armenia,Europe & Central Asia,Lower middle income
